In [1]:
import numpy as np
from skopt import BayesSearchCV
from tensorflow.keras.datasets import cifar10 
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")
from skopt.plots import plot_convergence
import matplotlib.pyplot as plt

In [2]:
# CIFAR-10 데이터 불러오기
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 데이터 전처리
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 학습 및 검증 데이터 분리
train_data, val_data, train_labels, val_labels = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

#한번 모양 확인하기
print(train_data.shape)
print(train_labels.shape)
print(val_data.shape)
print(val_labels.shape)


(40000, 32, 32, 3)
(40000, 10)
(10000, 32, 32, 3)
(10000, 10)


In [3]:
class VGGCVClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, learning_rate=1e-3, dropout_rate=0.0):
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate
        self.model = self._build_model()

    def _build_model(self):
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
        x = Flatten()(base_model.output)
        x = Dense(256, activation='relu')(x)
        x = Dropout(self.dropout_rate)(x)
        predictions = Dense(10, activation='softmax')(x)
        model = Model(inputs=base_model.input, outputs=predictions)

        model.compile(optimizer=Adam(learning_rate=self.learning_rate),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        return model
        model.summary

    
    def fit(self, X, y, val_data=None, val_labels=None):
        
        y_categorical = to_categorical(y, 10)

         # Early stopping 추가
        early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


        # 모델 컴파일 및 학습
        self.model.compile(optimizer=self.model.optimizer,
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])

        
        #모델 학습
        self.model.fit(X, y_categorical, epochs=5, batch_size=32,
                       validation_data=(val_data, val_labels), callbacks=[early_stopping], verbose=0)

    
    def predict(self, X):
        return np.argmax(self.model.predict(X), axis=1)




In [4]:
# 이전에 사용한 하이퍼파라미터
previous_learning_rate = 1e-3
previous_dropout_rate = 0.0

# 이전에 사용한 모델 생성
previous_model = VGGCVClassifier(learning_rate=previous_learning_rate, dropout_rate=previous_dropout_rate)

# 이전 모델 학습
previous_model.fit(train_data, np.argmax(train_labels, axis=1), val_data=val_data, val_labels=val_labels)

# 이전 모델 테스트 데이터 정확도 계산
previous_test_predictions = previous_model.predict(x_test)
previous_test_accuracy = accuracy_score(np.argmax(y_test, axis=1), previous_test_predictions)
print("이전 모델 테스트 데이터 정확도:", previous_test_accuracy)


1
2


KeyboardInterrupt: 

In [5]:
# 베이지안 최적화를 위한 검색 공간 정의
search_space = {'learning_rate': (1e-6, 1e-2, 'log-uniform'),
                'dropout_rate': (0.0, 0.5)}

# 베이지안 최적화 수행
opt = BayesSearchCV(
    VGGCVClassifier(),
    search_spaces=search_space,
    n_iter=20,
    random_state=42
)

opt.fit(train_data, np.argmax(train_labels, axis=1), val_data=val_data, val_labels=val_labels)

# 최적의 하이퍼파라미터 얻기
optimal_params = opt.best_params_
print("최적의 매개변수:", opt.best_params_)


KeyboardInterrupt: 

In [ ]:
# 최적화된 모델 생성
optimized_model = VGGCVClassifier(learning_rate=optimal_params['learning_rate'], dropout_rate=optimal_params['dropout_rate'])

# 최적화된 모델 학습
optimized_model.fit(train_data, np.argmax(train_labels, axis=1), val_data=val_data, val_labels=val_labels)

# 최적화된 모델 테스트 데이터 정확도 계산
optimized_test_predictions = optimized_model.predict(x_test)
optimized_test_accuracy = accuracy_score(np.argmax(y_test, axis=1), optimized_test_predictions)

print("최적화된 모델 테스트 데이터 정확도:", optimized_test_accuracy)


In [ ]:
# 테스트 데이터에 대한 정확도 계산
test_predictions = opt.predict(x_test)
test_accuracy = accuracy_score(np.argmax(y_test, axis=1), test_predictions)

print("테스트 데이터 정확도:", test_accuracy)

# 수렴 그래프 플로팅
plot_convergence(opt)
plt.show()
